In [1]:
import numpy as np
import random
import platform
import sys
import os
import math
import time
import matplotlib
from matplotlib import pyplot as plt
import vtk

In [2]:
raw_data_file = '/home/soumya/Test_DataSet/multivar_sampling_test_data/Isabel_pressure_velocity_qvapor.vti'
sampled_file = 'isabel_sampled.vtp'
var_name = 'Pressure'
var_name1 = 'Pressure_smooth'
query_th1 = -50
query_th2 = -5000

In [3]:
## load data

reader = vtk.vtkXMLImageDataReader()
#reader = vtk.vtkGenericDataObjectReader()
reader.SetFileName(raw_data_file)
reader.Update()
raw_data = reader.GetOutput()

reader1 = vtk.vtkXMLPolyDataReader()
reader1.SetFileName(sampled_file)
reader1.Update()
sampled_pts = reader1.GetOutput()

In [4]:
def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    index = x + dimx*(y+dimy*z)
    return index

In [5]:
extent = raw_data.GetExtent()
spacing = raw_data.GetSpacing()
origin = raw_data.GetOrigin()
dims = raw_data.GetDimensions()

query_pts = vtk.vtkPoints()
query_pts_physical = vtk.vtkPoints()
query_arr = vtk.vtkDoubleArray()
query_arr.SetName(var_name)

for i in range(sampled_pts.GetNumberOfPoints()):
    vo2 = sampled_pts.GetPointData().GetArray(var_name).GetTuple1(i)
    #if vo2 >= query_th1 and vo2 <= query_th2:
    if vo2 >= query_th2 and vo2 <= query_th1:    
        pts = sampled_pts.GetPoint(i)        
        ii = int((pts[0] - origin[0])/spacing[0] + 0.5)
        jj = int((pts[1] - origin[1])/spacing[1] + 0.5)
        kk = int((pts[2] - origin[2])/spacing[2] + 0.5)
        query_pts.InsertNextPoint([ii,jj,kk])
        query_arr.InsertNextTuple1(vo2)   
        query_pts_physical.InsertNextPoint(pts)
        
print (query_pts.GetNumberOfPoints())  

pdata = vtk.vtkPolyData()
pdata.SetPoints(query_pts_physical)
pdata.GetPointData().AddArray(query_arr)

writer1 = vtk.vtkXMLPolyDataWriter()
writer1.SetInputData(pdata)
writer1.SetFileName('recon_query.vtp')
writer1.Write()  

37905


1

In [6]:
new_data = vtk.vtkImageData()
new_data.SetDimensions(dims)
new_data.SetSpacing(spacing)
new_data.SetOrigin(origin)

query_arr_new = vtk.vtkDoubleArray()
query_arr_new.SetName(var_name)
query_arr_new.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

query_arr_new1 = vtk.vtkDoubleArray()
query_arr_new1.SetName(var_name1)
query_arr_new1.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

for i in range(dims[0]*dims[1]*dims[2]):
    query_arr_new.SetTuple1(i,0.0)
    query_arr_new1.SetTuple1(i,0.0)
             
for p in range(query_pts.GetNumberOfPoints()):   
    pts = query_pts.GetPoint(p)
    val = query_arr.GetTuple1(p)
    index = compute_3d_to_1d_map(int(pts[0]),int(pts[1]),int(pts[2]),dims[0],dims[1],dims[2])
    query_arr_new.SetTuple1(index,val)
    #query_arr_new.SetTuple1(index,1.0)
    
new_data.GetPointData().AddArray(query_arr_new) 

0

In [12]:
for k in range(dims[2]-1):
    for j in range(dims[1]-1):
        for i in range(dims[0]-1):
            
            index = compute_3d_to_1d_map(i,j,k,dims[0],dims[1],dims[2])
            val = query_arr_new.GetTuple1(index)
            
            sum_val = 0.0
            if val == 0.0:
                index1 = compute_3d_to_1d_map(i-1,j,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i+1,j,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j-1,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j+1,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j,k-1,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j,k+1,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                sum_val = sum_val / 8.0
                
                query_arr_new1.SetTuple1(index,sum_val)
                                
            else:
                query_arr_new1.SetTuple1(index,val)

new_data.GetPointData().AddArray(query_arr_new1)                

1

In [8]:
# window=1
# for k in range(dims[2]-window):
#     for j in range(dims[1]-window):
#         for i in range(dims[0]-window):
            
#             index = compute_3d_to_1d_map(i,j,k,dims[0],dims[1],dims[2])
#             val = query_arr_new.GetTuple1(index)
            
#             sum_val = 0.0
#             if val == 0.0:
                
#                 count=0
#                 for kk in range(k-window,k+window,1):
#                     for jj in range(j-window,j+window,1):
#                         for ii in range(i-window,i+window,1):

#                             index1 = compute_3d_to_1d_map(ii,jj,kk,dims[0],dims[1],dims[2])
#                             sum_val = sum_val + query_arr_new.GetTuple1(index1)
#                             count=count+1
                            
#                 sum_val = sum_val / count
                
#                 query_arr_new1.SetTuple1(index,sum_val)
                        
#             else:
#                 query_arr_new1.SetTuple1(index,val)

# new_data.GetPointData().AddArray(query_arr_new1)                        

In [13]:
writer = vtk.vtkXMLImageDataWriter()
writer.SetInputData(new_data)
writer.SetFileName('recon_query.vti')
writer.Write()           

1